In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler as SS
from joblib import dump
from config_store import targetDataSet, randomSeed
from utility_functions import dataSetSplit, doGridSearch, score

In [2]:
# Import and preview data set
data = pd.read_csv(targetDataSet)
data.head()

,Website,Packets,Bytes,Packets A → B,Bytes A → B,Packets B → A,Bytes B → A,Duration,Bits/s A → B,Bits/s B → A
0,0,1,712,1,712,0,0,16.763719,339,0
1,0,1,718,1,718,0,0,10.784664,532,0
2,0,1,889,1,889,0,0,4.927569,1443,0
3,0,1,712,1,712,0,0,13.479684,422,0
4,0,1,718,1,718,0,0,12.706807,452,0


In [3]:
# Separate data set using test/train split
XTrain, XTest, yTrain, yTest = dataSetSplit(data.drop(columns = ["Website"]).values, data["Website"].values)

# Scale data
ss = SS()
XTrainScaled = ss.fit_transform(XTrain)
XTestScaled = ss.transform(XTest)

In [4]:
# Define top 3 models to use in ada
dtc = DTC(random_state = randomSeed, min_samples_split = 6)
lr = LR(random_state = randomSeed, n_jobs = -1)
sgd = SGDClassifier(random_state = randomSeed, n_jobs = -1)

# Ada - DTC

In [5]:
# Perform hyperparameter optimization
# Define hyperparameters to check
hyperparameters = dict(n_estimators = np.array(range(50, 160, 10)), learning_rate = np.array(range(0, 10)))

# Set up and perform grid search
grid = doGridSearch(AdaBoostClassifier(estimator = dtc, random_state = randomSeed), hyperparameters, XTrain, yTrain)

c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
110 fits failed out of a total of 1100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
110 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\okisn\anaconda3\Lib\site-packa

In [6]:
# Check grid search results
gridDataframe = pd.DataFrame(grid.cv_results_)[["param_n_estimators", "param_learning_rate", "mean_test_score", "rank_test_score"]]
gridDataframe

,param_n_estimators,param_learning_rate,mean_test_score,rank_test_score
0,50,0,NaN,100
1,60,0,NaN,100
2,70,0,NaN,100
3,80,0,NaN,100
4,90,0,NaN,100
...,...,...,...,...
105,110,9,0.841402,31
106,120,9,0.841402,31
107,130,9,0.841402,31
108,140,9,0.841402,31


In [7]:
# Show best hyperparameter set
bestHyperSet = gridDataframe[gridDataframe["rank_test_score"] <= 1]
bestHyperSet

,param_n_estimators,param_learning_rate,mean_test_score,rank_test_score
72,110,6,0.853902,1


In [8]:
# Final model
adaDTC = AdaBoostClassifier(estimator = dtc, random_state = randomSeed, n_estimators = bestHyperSet["param_n_estimators"].iloc(0)[0], learning_rate = bestHyperSet["param_learning_rate"].iloc(0)[0])
adaDTC.fit(XTrain, yTrain)

AdaBoostClassifier(estimator=DecisionTreeClassifier(min_samples_split=6,
                                                    random_state=201),
                   learning_rate=6, n_estimators=110, random_state=201)

In [9]:
# Score model
score(adaDTC, XTrain, yTrain, XTest, yTest)

Training score: 0.9876237623762376
Testing score: 0.7450980392156863


In [10]:
# Save model
dump(adaDTC, "models/ada_dtc.joblib")

['models/ada_dtc.joblib']

# Ada - Log Reg

In [11]:
# Perform hyperparameter optimization
# Set up and perform grid search
grid = doGridSearch(AdaBoostClassifier(estimator = lr, random_state = randomSeed), hyperparameters, XTrainScaled, yTrain)

c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
110 fits failed out of a total of 1100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
110 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\okisn\anaconda3\Lib\site-packa

In [12]:
# Check grid search results
gridDataframe = pd.DataFrame(grid.cv_results_)[["param_n_estimators", "param_learning_rate", "mean_test_score", "rank_test_score"]]
gridDataframe

,param_n_estimators,param_learning_rate,mean_test_score,rank_test_score
0,50,0,NaN,100
1,60,0,NaN,100
2,70,0,NaN,100
3,80,0,NaN,100
4,90,0,NaN,100
...,...,...,...,...
105,110,9,0.274695,41
106,120,9,0.286890,29
107,130,9,0.282012,33
108,140,9,0.274695,41


In [13]:
# Show best hyperparameter set
bestHyperSet = gridDataframe[gridDataframe["rank_test_score"] <= 1]
bestHyperSet

,param_n_estimators,param_learning_rate,mean_test_score,rank_test_score
22,50,2,0.470549,1


In [14]:
# Final model
adaLR = AdaBoostClassifier(estimator = lr, random_state = randomSeed, n_estimators = bestHyperSet["param_n_estimators"].iloc(0)[0], learning_rate = bestHyperSet["param_learning_rate"].iloc(0)[0])
adaLR.fit(XTrainScaled, yTrain)

AdaBoostClassifier(estimator=LogisticRegression(n_jobs=-1, random_state=201),
                   learning_rate=2, random_state=201)

In [15]:
# Score model
score(adaLR, XTrainScaled, yTrain, XTestScaled, yTest)

Training score: 0.5024752475247525
Testing score: 0.5098039215686274


In [16]:
# Save model
dump(adaLR, "models/ada_log_reg.joblib")

['models/ada_log_reg.joblib']

# Ada - SGD

In [17]:
# Perform hyperparameter optimization
# Set up and perform grid search
grid = doGridSearch(AdaBoostClassifier(estimator = sgd, random_state = randomSeed, algorithm = "SAMME"), hyperparameters, XTrainScaled, yTrain)

c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
110 fits failed out of a total of 1100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
110 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\okisn\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\okisn\anaconda3\Lib\site-packa

In [18]:
# Check grid search results
gridDataframe = pd.DataFrame(grid.cv_results_)[["param_n_estimators", "param_learning_rate", "mean_test_score", "rank_test_score"]]
gridDataframe

,param_n_estimators,param_learning_rate,mean_test_score,rank_test_score
0,50,0,NaN,100
1,60,0,NaN,100
2,70,0,NaN,100
3,80,0,NaN,100
4,90,0,NaN,100
...,...,...,...,...
105,110,9,0.212683,97
106,120,9,0.227561,80
107,130,9,0.217683,94
108,140,9,0.212561,98


In [19]:
# Show best hyperparameter set
bestHyperSet = gridDataframe[gridDataframe["rank_test_score"] <= 1]
bestHyperSet

,param_n_estimators,param_learning_rate,mean_test_score,rank_test_score
11,50,1,0.408659,1
12,60,1,0.408659,1
13,70,1,0.408659,1
14,80,1,0.408659,1
15,90,1,0.408659,1
16,100,1,0.408659,1
17,110,1,0.408659,1
18,120,1,0.408659,1
19,130,1,0.408659,1
20,140,1,0.408659,1


In [20]:
# Final model
adaSGD = AdaBoostClassifier(estimator = sgd, random_state = randomSeed, algorithm = "SAMME", n_estimators = bestHyperSet["param_n_estimators"].iloc(0)[0], learning_rate = bestHyperSet["param_learning_rate"].iloc(0)[0])
adaSGD.fit(XTrainScaled, yTrain)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=SGDClassifier(n_jobs=-1, random_state=201),
                   learning_rate=1, random_state=201)

In [21]:
# Score model
score(adaSGD, XTrainScaled, yTrain, XTestScaled, yTest)

Training score: 0.5445544554455446
Testing score: 0.49019607843137253


In [22]:
# Save model
dump(adaSGD, "models/ada_sgd.joblib")

['models/ada_sgd.joblib']